# 1 Include the libraries

In [15]:
from __future__ import division
import pandas as pd
pd.set_option('max_colwidth', 800)
import MySQLdb
import urllib2
import datetime
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from bokeh.charts import Bar, output_notebook, show
import json
import re

## include self_written fuctions
from read_sql_table import *
from read_survey_question import *
from read_survey_response import *
from read_json_response import *
from read_json_weights import *
from computing_each_question import *
from plot_geo_address import *
from plot_time_serial import *
from computing_survey_metrics import *
from computing_open_text import *
from clear_format_tags import *

In [16]:
import ipywidgets
from ipywidgets import *
from IPython.display import display

# 2 Set up the survey variables


>  SurveyId = 141  
>  start_time = pd.tslib.Timestamp('2016-09-01 00:00:00')  
>  end_time = pd.tslib.Timestamp(datetime.datetime.utcnow())  
>  repeatable = False   


In [17]:
start_time = pd.tslib.Timestamp('2017-04-07 00:00:00')
# start_time = pd.tslib.Timestamp('2017-04-29 00:00:00')
end_time = pd.tslib.Timestamp(datetime.datetime.utcnow())
repeatable = False

# 3 Get all the survey responses from the first screener

In [18]:
SurveyId_1 = 23
SurveyId_2 = 229
SurveyId_3 = 234

In [19]:
survey_questions = get_survey_question(SurveyId_1)

survey_response_unique = get_survey_response(SurveyId_1, start_time, end_time, repeatable)
result_survey_1 = get_json_response(survey_questions, survey_response_unique['jsonResponse'].tolist())
result_survey_1 = pd.concat([result_survey_1, survey_response_unique.drop(['jsonResponse'], axis = 1)], axis = 1)

# calculate the weights and total_score for a screener
weights = get_json_weight(survey_questions, survey_response_unique['jsonResponse'].tolist())
result_survey_1['total_score_for_screener'] = weights.sum(axis = 1)

result_survey_1, survey_questions = clear_html_tag(result_survey_1, survey_questions)

In [20]:
survey_questions = get_survey_question(SurveyId_2)

survey_response_unique = get_survey_response(SurveyId_2, start_time, end_time, repeatable)
result_survey_2 = get_json_response(survey_questions, survey_response_unique['jsonResponse'].tolist())
result_survey_2 = pd.concat([result_survey_2, survey_response_unique.drop(['jsonResponse'], axis = 1)], axis = 1)

# calculate the weights and total_score for a screener
weights = get_json_weight(survey_questions, survey_response_unique['jsonResponse'].tolist())
result_survey_2['total_score_for_screener'] = weights.sum(axis = 1)

result_survey_2, survey_questions = clear_html_tag(result_survey_2, survey_questions)

In [21]:
survey_questions = get_survey_question(SurveyId_3)

survey_response_unique = get_survey_response(SurveyId_3, start_time, end_time, repeatable)
result_survey_3 = get_json_response(survey_questions, survey_response_unique['jsonResponse'].tolist())
result_survey_3 = pd.concat([result_survey_3, survey_response_unique.drop(['jsonResponse'], axis = 1)], axis = 1)

# calculate the weights and total_score for a screener
weights = get_json_weight(survey_questions, survey_response_unique['jsonResponse'].tolist())
result_survey_3['total_score_for_screener'] = weights.sum(axis = 1)

result_survey_3, survey_questions = clear_html_tag(result_survey_3, survey_questions)

# 4 Merge all the results

In [22]:
result_temp = pd.merge(result_survey_1, result_survey_2, on ='clientToken', how = 'outer')
result = pd.merge(result_temp, result_survey_3, on = 'clientToken', how = 'outer')

In [23]:
b1 = Button(description='Adjust the date')
b1.button_style = 'info'
display(b1)

start_year = IntSlider(value = 2017, min = 2016, max = 2018, step = 1, description = 'S_Year')
start_month = IntSlider(value = 1, min = 1, max = 12, step = 1, description = 'S_Month')
start_day = IntSlider(value = 1, min = 1, max = 31, step = 1, description = 'S_Day')

end_year = IntSlider(value = 2017, min = 2016, max = 2018, step = 1, description = 'E_Year')
end_month = IntSlider(value = 5, min = 1, max = 12, step = 1, description = 'E_Month')
end_day = IntSlider(value = 1, min = 1, max = 31, step = 1, description = 'E_Day')

w = ipywidgets.ToggleButtons(
    options=['data', 'metrics'],
    description='options',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)


@interact(sd1 = start_year, sd2 = start_month, sd3 = start_day, \
         ed1 = end_year, ed2 = end_month, ed3 = end_day, w = w, __manual = True )
def click_me(sd1, sd2, sd3, ed1, ed2, ed3, w = w):
    print sd1, sd2, sd3, ed1, ed2, ed3
    
    
    start_date = datetime.datetime(start_year.value, start_month.value, start_day.value)
    end_date = datetime.datetime(end_year.value, end_month.value, end_day.value)

    start_date = pd.tslib.Timestamp(start_date)
    end_date = pd.tslib.Timestamp(end_date)
    if start_date > end_time:
        raise ValueError('The start time is later than the earliest response')
    elif start_date > pd.tslib.Timestamp(datetime.datetime.utcnow()):
        raise ValueError('The start time is later than the current time')
    elif end_date < start_time:
        raise ValueError('The end time is before the survey start time')
        
    filter = [False] * len(result)
    for i in xrange(len(result)):
        if result['startTime_x'][i] > start_date and result['startTime_x'][i] < end_date:
            filter[i] = True
    temp_result = result[filter]
    
    if w == 'data':
        print 'The raw data is created'
        display(temp_result)
        result.to_excel('connetic_%s.xlsx' % end_time)
    elif w == 'metrics':
        print "\nsurvey metrics for survey: %d" % SurveyId_1
        display(get_survey_metrics(SurveyId_1, start_date, end_date, result_survey_1))

        print "\nsurvey metrics for survey: %d" % SurveyId_2
        display(get_survey_metrics(SurveyId_2, start_date, end_date, result_survey_2))

        print "\nsurvey metrics for survey: %d" % SurveyId_3
        display(get_survey_metrics(SurveyId_3, start_date, end_date, result_survey_3))
    
    

In [24]:

# @interact(w = w, __manual = True)
# def click_me(w):
#     if w == 'data':
#         print 'The raw data is created'
#         result.to_excel('connetic_%s.xlsx' % end_time)
#     elif w == 'metrics':
#         print "\nsurvey metrics for survey: %d" % SurveyId_1
#         display(get_survey_metrics(SurveyId_1, start_time, end_time, result_survey_1))

#         print "\nsurvey metrics for survey: %d" % SurveyId_2
#         display(get_survey_metrics(SurveyId_2, start_time, end_time, result_survey_2))

#         print "\nsurvey metrics for survey: %d" % SurveyId_3
#         display(get_survey_metrics(SurveyId_3, start_time, end_time, result_survey_3))

In [25]:
# result.to_excel('connetic_May02.xlsx')

# 5 Get survey metrics

In [26]:
# print "survey metrics for survey: %d" % SurveyId_1
# get_survey_metrics(SurveyId_1, start_time, end_time, result_survey_1)

# print "survey metrics for survey: %d" % SurveyId_2
# get_survey_metrics(SurveyId_2, start_time, end_time, result_survey_2)

# print "survey metrics for survey: %d" % SurveyId_3
# get_survey_metrics(SurveyId_3, start_time, end_time, result_survey_3)